The stopping power of a material in J/m is $S(E)$. Thus we have 
\begin{equation}
\begin{split}
\frac{dE}{dx} = S(E)\\
E = \int S(E) dx
\end{split}
\end{equation}
How do I write $S(E)$ in terms of $x$? I would have to first find $E(x)$
\begin{equation}
\begin{split}
\frac{dE(x)}{dx} = S(E(x))\\
E(x) = \int S(E(x)) dx\\
\end{split}
\end{equation}
We define $E(x=0) = E_0$ which gives
\begin{equation}
\begin{split}
\frac{dE(x=0)}{dx} = S(E_0)\\
E(x=dx) = E_0 + S(E_0)\\
\frac{dE(x=dx)}{dx} = S(E(x=dx)) = S(E_0+S(E_0))\\
E(x=2dx) = E(x=dx) + S(E(x=dx)) = E_0 + S(E_0) + S(E_0+S(E_0))\\
\end{split}
\end{equation}





I now define $E(x=\alpha) = E_\alpha$. 
Now I will use induction to prove the following recursive formula for $E(x=ndx)$ where $n\in\mathbb{Z}, n>0$
\begin{equation}
\begin{split}
E_{ndx} = f_n\\
f_1 = E_0 + S(E_0)\\
f_n = f_{n-1} + S(f_{n-1}) 
\end{split}
\end{equation}
We have already shown that it holds for $n=1$ and $n=2$, so now we will show that if it holds for $n$, then it must also hold for $n+1$. Thus we have
\begin{equation}
\begin{split}
E_{ndx} = f_n\\
E_{(n+1)dx} = \frac{dE_{ndx}}{dx} + E_{ndx}\\
E_{(n+1)dx} = \frac{dE_{ndx}}{dx} + f_n\\
E_{(n+1)dx} = S(f_n) + f_n = f_{n+1}\\
\end{split}
\end{equation}

